In [15]:
# Step 1: Install required libraries
!pip install transformers rouge-score nltk bert-score

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [16]:
# Import necessary libraries
import json
from transformers import pipeline
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import bert_score
import random

In [17]:
# Step 3: Load the dataset
# Make sure to upload your dataset to Kaggle and use the correct path
input_file_path = '/kaggle/input/nuclear/cleaned_json.json'

In [18]:
# Load the dataset from the JSON file
with open(input_file_path, "r") as f:
    data = json.load(f)

In [20]:
# Check available keys in the entries
print("Available keys in each entry:", data["Sheet1"][0].keys())

Available keys in each entry: dict_keys(['Document', 'Content '])


In [21]:
# Adjust the key according to the correct structure of the JSON file
content_key = "Content "  # Adjust according to the correct key in your file

In [22]:
# Load pre-trained models for question generation and question answering
question_generator = pipeline("text2text-generation", model="valhalla/t5-small-qg-hl")
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [25]:
# Initialize the Rouge scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

In [26]:
output_data = []

In [27]:
# Generate questions and answers
for entry in data["Sheet1"]:
    # Check if the key exists in the entry
    if content_key not in entry:
        print(f"Key '{content_key}' not found in entry: {entry}")
        continue

    context = entry[content_key]
    print(f"Processing context: {context[:100]}...")  # Display the first 100 characters of the context

    try:
        # Generate 10 distinct questions
        questions = question_generator(f"generate question: {context}", num_return_sequences=10, num_beams=10, max_length=50)
        generated_questions = [q['generated_text'] for q in questions]
    except Exception as e:
        print(f"Error generating questions: {e}")
        continue

    for generated_question in generated_questions:
        print(f"Generated question: {generated_question}")

        # Ensure the generated question is complete
        if not generated_question.endswith('?'):
            generated_question += "?"

        try:
            # Generate answer with a higher length limit
            generated_answer = qa_pipeline(question=generated_question, context=context, max_answer_len=200)['answer']
            print(f"Generated answer: {generated_answer}")
        except Exception as e:
            print(f"Error generating answer: {e}")
            continue

        output_data.append({
            "prompt": generated_question,
            "response": generated_answer,
        })

        # Check if we have reached 100 questions
        if len(output_data) >= 100:
            break

    # Check if we have reached 100 questions
    if len(output_data) >= 100:
        break

Token indices sequence length is longer than the specified maximum sequence length for this model (735 > 512). Running this sequence through the model will result in indexing errors


Processing context: There are seven licensed nuclear power plant (NPP) sites in Canada, operated by four different  lice...
Generated question: What is the CNSC's Integrated Safety Assessment of Canadian Nuclear Power Plants for 2009?
Generated answer: NPP Report
Generated question: What is the CNSC's Integrated Safety Assessment of Canadian Nuclear Power Plants?
Generated answer: NPP Report
Generated question: What is the CNSC's Integrated Safety Assessment of Canadian Nuclear Power Plants report?
Generated answer: NPP Report
Generated question: What is the CNSC's Integrated Safety Assessment of Canadian Nuclear Power Plants report based on?
Generated answer: findings made throughout the year during  inspections, desktop reviews, event reviews and reviews of performance indicators
Generated question: What is the CNSC's Integrated Safety Assessment of Canadian Nuclear Power Plants in 2009?
Generated answer: NPP Report) assesses the safety performance
Generated question: What is the CNS

In [28]:
# Save the output data to a JSON file
output_file_path = '/kaggle/working/output.json'
with open(output_file_path, "w") as f:
    json.dump(output_data, f)

In [29]:
print(f"Output saved to {output_file_path}")

Output saved to /kaggle/working/output.json
